In [161]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import uniform
import tqdm
import math

In [162]:
# CI function
def CI(data,alpha):
    sample_mean=np.mean(data) # data is a list!
    sample_sigma=np.std(data)
    critical_point = norm.ppf(1-alpha/2)
    LB=sample_mean-critical_point*sample_sigma/np.sqrt(len(data))
    UB=sample_mean+critical_point*sample_sigma/np.sqrt(len(data))
    return LB,UB

# Question 1 :

In [148]:
T,N=5,10
r=0.04
S=0.014
size=100000
default_rate=np.array([0.05, 0.01, 0.05, 0.05, 0.01, 0.1, 0.01, 0.09, 0.1, 0.02])
recovery_rate=np.array([0.1, 0.1, 0.3, 0.1, 0.3, 0.1, 0.2, 0.2, 0.1, 0.1])

# Copula matrix
Copula=np.ones((N,N))*0.2
for i in range(N):
    Copula[i,i]=1
#Generate Random variables
mean=np.zeros(N)
payoff=[]
for i in range(size):
    # Step 1:
    Z=np.random.multivariate_normal(mean, Copula)
    U=norm.cdf(Z)
    default_time=-np.log(1-U)/default_rate
    
    # Step 2:
    fifth_indicator=np.where(default_time==sorted(default_time)[4])[0][0]
    tao=default_time[fifth_indicator]
    
    # Step 3:
    if tao>T:
        V_prot=S*np.exp(-r)*(1-np.exp(-5*r))/(1-np.exp(-r))
        V_value=0
    else:
        Tj=math.floor(tao)
        V_prot=S*np.exp(-r)*(1-np.exp(-Tj*r))/(1-np.exp(-r))+S*np.exp(-r*tao)*(tao-Tj)
        V_value=(1-recovery_rate[fifth_indicator])*np.exp(-r*tao)
    payoff.append(V_value-V_prot)
print(np.mean(payoff),CI(payoff,0.05))

0.0009861596131251914 (-0.00031977871038803155, 0.0022920979366384143)


# Question 2 :

In [189]:
'''
Exit time T follow pdf f(x)=x/18 0<x<6
If T<6, Exit payoff: p*IPO payoff+(1-p)*M&A payoff 
Exit value above 1B would be IPO, all other are M&A
If T=6, exit would be M&A
Series B / M&A payoff = formula 1
Series B / IPO payoff = X*Ib/Pb
Also need to check if XT>1000 or not, 
'''
# parameters
Pa,Pb,Ia,Ib=450,1000,50,100
share_new,share_old,share_current=0.1,0.1,0.8
r,sigma=0.025,0.9
size=10000000

X0=673 ##

U=np.random.uniform(0,1,size)
T=np.sqrt(36*U)

Z=np.random.normal(0,1,size)
X=X0*np.exp((r-0.5*(sigma**2))*T+sigma*np.sqrt(T)*Z)


# Merge = np.max([np.min([Ia*X[np.where(X<1000)]/(Ia+Ib),np.ones(len(np.where(X<1000)[0]))*Ib],axis=0),X[np.where(X<1000)]*Ib/Pb],axis=0)
Merge=np.maximum(np.minimum(Ib*X[X<1000]/(Ia+Ib),np.ones(len(X[X<1000]))*Ib),X[X<1000]*Ib/Pb)*np.exp(-r*T[X<1000])
IPO = (X[X>=1000]*Ib/Pb)*np.exp(-r*T[X>=1000])
payoff=np.append(Merge,IPO)

print(np.mean(payoff),CI(payoff,0.05))



100.09854922074207 (99.82034175838102, 100.37675668310312)
